# [Module 2.5] Prophet 및 DeepAR+ Forecast 생성 (Create a Forecast)

이전 모듈에서 생성한 Predictor를 가지고 Prophet 및 DeepAR+ Forecast 생성을 합니다. <br>
Foreast의 의미는 실질적으로 전체 학습을 다시하고 서비스를 할 수 있는 준비를 하는 과정 입니다. <br>
Forecast의 생성 이후에 실제 미래의 예측(Inference)을 일괄적으로 하여 S3에 저장을 합니다.

---
이 과정은 약 40분 소요 되니다. (About 40 mins may be elapsed)

In [4]:
import boto3
from time import sleep
import pandas as pd
import json
import time
import pprint
import numpy as np

In [5]:
%store -r

In [6]:
session = boto3.Session(region_name=region)
forecast = session.client(service_name='forecast')

## Create Prophet and DeepAR+ Campaign

In [5]:
# Prophet
item_meta_prophet_forecastName = project+'_prophet_forecast' + item_meta_suffix + suffix
prophet_create_forecast_response=forecast.create_forecast(
    ForecastName = item_meta_prophet_forecastName,
    PredictorArn = item_meta_prophet_predictor_arn)


In [6]:
item_meta_prophet_forecast_arn = prophet_create_forecast_response['ForecastArn']
forecast.describe_forecast(ForecastArn = item_meta_prophet_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_prophet_forecast_ItemMeta91891',
 'ForecastName': 'WalmartKaggleWithThreeDatasets_prophet_forecast_ItemMeta91891',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:057716757052:predictor/WalmartKaggleWithThreeDatasets_prophet_algo_1_ItemMeta91891',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:057716757052:dataset-group/WalmartKaggleWithThreeDatasetsDSG_ItemMeta91891',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 8, 23, 8, 29, 22, 965000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 23, 8, 29, 22, 965000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': '4e1ee747-3d4f-4ea6-ac3e-d0ce7a52168d',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 23 Aug 2020 08:29:24 GMT',
   'x-amzn-requestid': '4e1ee747-3d4f-4ea6-ac3e-d0ce7a52168d

In [9]:
# DeepAR+
item_meta_deeparp_forecastName = project+'_deeparp_forecast' + item_meta_suffix + suffix
deeparp_create_forecast_response=forecast.create_forecast(
    ForecastName = item_meta_deeparp_forecastName,
    PredictorArn = item_meta_deepar_predictor_arn)


In [10]:
item_meta_deeparp_forecast_arn = deeparp_create_forecast_response['ForecastArn']
forecast.describe_forecast(ForecastArn = item_meta_deeparp_forecast_arn)

{'ForecastArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_deeparp_forecast_ItemMeta91891',
 'ForecastName': 'WalmartKaggleWithThreeDatasets_deeparp_forecast_ItemMeta91891',
 'ForecastTypes': ['0.1', '0.5', '0.9'],
 'PredictorArn': 'arn:aws:forecast:ap-northeast-2:057716757052:predictor/WalmartKaggleWithThreeDatasets_deeparp_algo_1_ItemMeta91891',
 'DatasetGroupArn': 'arn:aws:forecast:ap-northeast-2:057716757052:dataset-group/WalmartKaggleWithThreeDatasetsDSG_ItemMeta91891',
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 8, 23, 8, 29, 41, 451000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 23, 8, 29, 41, 451000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'c551f67a-808c-4b62-aff7-6a451958e2fb',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 23 Aug 2020 08:29:41 GMT',
   'x-amzn-requestid': 'c551f67a-808c-4b62-aff7-6a451958e2fb

In [11]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast(ForecastArn= item_meta_prophet_forecast_arn)['Status']
    createDeeparpStatus = forecast.describe_forecast(ForecastArn= item_meta_deeparp_forecast_arn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREA

## Upload forecast results to S3

위의 과정이 세션이 끊기어서 forecast arn을 가져오지 못할 경우에는 Forecast 콘솔에 가서 Forecast arn을 복사하여 아래에 넣고 주석 제거 후 실행 하세요

In [1]:
# item_meta_prophet_forecast_arn = 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_prophet_forecast_ItemMeta91891'
# item_meta_deeparp_forecast_arn = 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_deeparp_forecast_ItemMeta91891'

In [7]:
meta_data_prophet_path = "s3://" + bucket_name + "/" + bucket_folder + "/prophet_" + item_meta_suffix + suffix + "/" 
meta_data_prophet_job_name = "ProphetExport1" + item_meta_suffix + suffix
create_forecast_export_job_prophet_response = forecast.create_forecast_export_job(
    ForecastExportJobName = meta_data_prophet_job_name,
    ForecastArn = item_meta_prophet_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": meta_data_prophet_path,
            "RoleArn": role_arn
        }
    })


In [13]:
ItemMetaForecastProphetExportJobArn = create_forecast_export_job_prophet_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ItemMetaForecastProphetExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast-export-job/WalmartKaggleWithThreeDatasets_prophet_forecast_ItemMeta91891/ProphetExport1_ItemMeta91891',
 'ForecastExportJobName': 'ProphetExport1_ItemMeta91891',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_prophet_forecast_ItemMeta91891',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-057716757052/WalmartKaggleWithThreeDatasets/prophet__ItemMeta91891',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart91891'}},
 'Status': 'ACTIVE',
 'CreationTime': datetime.datetime(2020, 8, 23, 10, 5, 17, 436000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 23, 10, 12, 10, 462000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'cc80f697-2d22-4f70-8ff2-146c1954cc28',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 23 Aug 2020 10:13:22 GMT',


In [9]:
item_meta_deeparp_path = "s3://" + bucket_name + "/" + bucket_folder  + "/deeparp_" + item_meta_suffix + suffix + "/" 
item_meta_deeparp_job_name = "DeepARPExport1" + item_meta_suffix + suffix
create_forecast_export_job_deepar_response = forecast.create_forecast_export_job(
    ForecastExportJobName=item_meta_deeparp_job_name,
    ForecastArn=item_meta_deeparp_forecast_arn,
    Destination={
        "S3Config" : {
            "Path": item_meta_deeparp_path,
            "RoleArn": role_arn
        }
    })

In [10]:
ItemMetaForecastDeeparExportJobArn = create_forecast_export_job_deepar_response["ForecastExportJobArn"]
forecast.describe_forecast_export_job(ForecastExportJobArn = ItemMetaForecastDeeparExportJobArn)

{'ForecastExportJobArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast-export-job/WalmartKaggleWithThreeDatasets_deeparp_forecast_ItemMeta91891/DeepARPExport1_ItemMeta91891',
 'ForecastExportJobName': 'DeepARPExport1_ItemMeta91891',
 'ForecastArn': 'arn:aws:forecast:ap-northeast-2:057716757052:forecast/WalmartKaggleWithThreeDatasets_deeparp_forecast_ItemMeta91891',
 'Destination': {'S3Config': {'Path': 's3://sagemaker-ap-northeast-2-057716757052/WalmartKaggleWithThreeDatasets/deeparp__ItemMeta91891',
   'RoleArn': 'arn:aws:iam::057716757052:role/ForecastRoleWalmart91891'}},
 'Status': 'CREATE_PENDING',
 'CreationTime': datetime.datetime(2020, 8, 23, 10, 5, 19, 852000, tzinfo=tzlocal()),
 'LastModificationTime': datetime.datetime(2020, 8, 23, 10, 5, 19, 852000, tzinfo=tzlocal()),
 'ResponseMetadata': {'RequestId': 'ae92f9bd-04e3-4428-9566-b7c1c8e6af71',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'content-type': 'application/x-amz-json-1.1',
   'date': 'Sun, 23 Aug 2020 10:05:20

In [11]:
%%time
# Check the Prophet status

while True:
    createProphetStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ItemMetaForecastProphetExportJobArn)['Status']
    createDeeparpStatus = forecast.describe_forecast_export_job(ForecastExportJobArn= ItemMetaForecastDeeparExportJobArn)['Status']    
    print("Prophet: ", createProphetStatus)
    print("DeepARP: ", createProphetStatus)          
    if createProphetStatus != 'ACTIVE' and createProphetStatus != 'CREATE_FAILED':
        sleep(60)
    elif createDeeparpStatus != 'ACTIVE' and createDeeparpStatus != 'CREATE_FAILED':
        sleep(60)
    else:
        break

Prophet:  CREATE_PENDING
DeepARP:  CREATE_PENDING
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  CREATE_IN_PROGRESS
DeepARP:  CREATE_IN_PROGRESS
Prophet:  ACTIVE
DeepARP:  ACTIVE
Prophet:  ACTIVE
DeepARP:  ACTIVE
CPU times: user 63.9 ms, sys: 3.41 ms, total: 67.3 ms
Wall time: 8min 1s


In [12]:
%store item_meta_prophet_forecast_arn
%store item_meta_deeparp_forecast_arn
%store ItemMetaForecastProphetExportJobArn
%store ItemMetaForecastDeeparExportJobArn



Stored 'item_meta_prophet_forecast_arn' (str)
Stored 'item_meta_deeparp_forecast_arn' (str)
Stored 'ItemMetaForecastProphetExportJobArn' (str)
Stored 'ItemMetaForecastDeeparExportJobArn' (str)
